# Dynamic Range Analysis

In [1]:
from __future__ import division, unicode_literals

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.models import ColumnDataSource, Label
from bokeh.plotting import figure
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, PreText
from ipywidgets import interact
from scipy.optimize import fsolve

import colour
import colour_hdri

In [2]:
output_notebook()

Loading BokehJS ...

## Helpers & Resources

In [3]:
BOKEH_TOOLS = 'pan,wheel_zoom,box_zoom,reset,resize'

BIT_DEPTH = 12
MINIMUM_REPRESENTABLE_VALUE = 1 / 2 ** BIT_DEPTH
MAXIMUM_REPRESENTABLE_VALUE = 1


def solve_EV_range(function,
                   minimum_representable_value=MINIMUM_REPRESENTABLE_VALUE, 
                   maximum_representable_value=MAXIMUM_REPRESENTABLE_VALUE, 
                   **kwargs):
    
    EV_min = fsolve(lambda x: function(2 ** x, **kwargs) - minimum_representable_value, 0)
    EV_max = fsolve(lambda x: function(2 ** x, **kwargs) - maximum_representable_value, 0)
    EV_neutral_gray = fsolve(lambda x: function(2 ** x, **kwargs) - 0.18, 0)
    EV_domain = np.arange(np.floor(EV_min), np.ceil(EV_max) + 1)
    
    EV_range  = function(2 ** EV_domain, **kwargs)
    
    return colour.Structure(
        **{'EV_domain': EV_domain.astype(np.int_), 
           'EV_range': EV_range,
           'EV_min': EV_min,
           'EV_max': EV_max,
           'EV_neutral_gray': EV_neutral_gray})


def dynamic_range_plot(function, 
                       minimum_representable_value=MINIMUM_REPRESENTABLE_VALUE, 
                       maximum_representable_value=MAXIMUM_REPRESENTABLE_VALUE, 
                       **kwargs):

    solve_r = solve_EV_range(
        function, 
        minimum_representable_value, 
        maximum_representable_value,
        **colour.filter_kwargs(function, **kwargs))

    settings = colour.Structure(
        **{'title': None,
           'width': 720,
           'height': 405,
           'x_range': None,
           'y_range': None,
           'y_axis_location' : 'right',
           'toolbar_location': 'left',
           'toolbar_sticky': False,
           'tools': BOKEH_TOOLS})
    settings.update({(key, value) for key, value in kwargs.items() 
                     if key in settings})

    plot = figure(**settings)

    plot.xaxis.axis_label = 'Dynamic Range (EV)'
    plot.xaxis.major_tick_line_width = 8
    
    EV_rectangle = plot.rect(
        solve_r.EV_domain, solve_r.EV_range, width=0.5, height=0.01)    
    
    neutral_gray_line = plot.line(
        (-64, 64), np.ones(2) * 0.18, line_width=2, line_dash=[8, 2])    

    data = {'Metric': ('Minimum EV',
                       'Maximum EV',
                       'EV < 18%',
                       'EV > 18%'),
            'Value':(solve_r.EV_min, 
                     solve_r.EV_max,
                     solve_r.EV_min - solve_r.EV_neutral_gray,
                     solve_r.EV_max - solve_r.EV_neutral_gray)}
    source = ColumnDataSource(data)
    columns = [TableColumn(field='Metric', title='Metric'),
               TableColumn(field='Value', title='Value')]
    table = DataTable(source=source, 
                      columns=columns, 
                      row_headers=False, 
                      width=int(settings.width / 3))
    
    handle = show(row(plot, table), notebook_handle=True)

    return colour.Structure(
        **{'EV_rectangle': EV_rectangle,
           'table': table,
           'handle': handle})


def update_dynamic_range_plot(structure, function, **kwargs):
    solve_r = solve_EV_range(function, **kwargs)
 
    structure.EV_rectangle.data_source.data['x'] = solve_r.EV_domain 
    structure.EV_rectangle.data_source.data['y'] = solve_r.EV_range

    structure.table.source.data = {
        'Metric': ('Minimum EV',
                   'Maximum EV',
                   'EV < 18%',
                   'EV > 18%'),
        'Value':(solve_r.EV_min, 
                 solve_r.EV_max,
                 solve_r.EV_min - solve_r.EV_neutral_gray,
                 solve_r.EV_max - solve_r.EV_neutral_gray)}

    push_notebook(handle=structure.handle)  

## Gamma

In [4]:
GAMMA_FUNCTION_S = dynamic_range_plot(
    colour.gamma_function, 
    x_range=(-BIT_DEPTH, 0), 
    y_range=(0, 1), 
    exponent=1)


def update_gamma_function(exponent=1.0):
    update_dynamic_range_plot(
        GAMMA_FUNCTION_S, colour.gamma_function, exponent=exponent)

In [5]:
interact(update_gamma_function, exponent=(0.1, 4, 0.01))

<function __main__.update_gamma_function>

## Exposure

In [6]:
EXPOSURE_FUNCTION_S = dynamic_range_plot(
    colour_hdri.adjust_exposure, 
    x_range=(-BIT_DEPTH, 0), 
    y_range=(0, 1), 
    EV=0)


def update_exposure_function(EV_a=0):
    update_dynamic_range_plot(
        EXPOSURE_FUNCTION_S, colour_hdri.adjust_exposure, EV=EV_a)

In [7]:
interact(update_exposure_function, EV_a=(-4, 4, 0.1))

<function __main__.update_exposure_function>

In [8]:
_ = dynamic_range_plot(
    colour_hdri.tonemapping_operator_simple, 
    x_range=(-BIT_DEPTH, 4), 
    y_range=(0, 1))